In [0]:
%load_ext autoreload
%autoreload 2

import matplotlib
from matplotlib import pyplot
pyplot.ion()
pyplot.style.use('dark_background')

import torch as th
from rl.algs.dqn import *

In [2]:
from rl.algs import experiment
from rl.algs import model
from rl.algs.envs import grid_world

BATCH_SIZE = 32
TRAIN_START_STEP = 3 * BATCH_SIZE
NUM_TRAIN_STEPS = 10000 # 17
REPLAY_SIZE = 100000
TARGET_NETWORK_UPDATE_FREQ = 500


env = environment.GridWorldWrapped(grid_shape=(4, 4, 1))
conv_specs = {
  model.ConvSpec(4, 1, 32),
}
fc_specs = (64,)
model_params = {
  'conv_specs': conv_specs,
  'fc_specs': fc_specs,
}

EPS_DECAY = util.Schedule((0, TRAIN_START_STEP, NUM_TRAIN_STEPS / 1, NUM_TRAIN_STEPS),
                          (1, 1, 0.0, 0.0))
EPS_RANDOM = util.Schedule((0,), (1.0,))
adam_params = {
  'fn': th.optim.Adam,
  'kwargs': {
    'lr': 0.005,
  }
}
rmsprop_params = {
  'fn': th.optim.RMSprop,
  'kwargs': {
    'lr': 0.00025,
    'momentum': 0.95,
    'alpha': 0.95,
    'eps': 0.01,
  }
}

policy_params = {
    'opt_params': rmsprop_params,
    'eps_sched': EPS_DECAY,  # TODO: change
    'target_update_freq': TARGET_NETWORK_UPDATE_FREQ,
}
exp = experiment.Experiment(env, model.QNetwork, model_params,
                            Dqn, policy_params)

print(exp.policy.model)

rb = ReplayBuffer(REPLAY_SIZE)
for step in range(TRAIN_START_STEP):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)

eps_rew = 0
for step in range(TRAIN_START_STEP, NUM_TRAIN_STEPS):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)
    
  experience_batch = rb.sample(BATCH_SIZE)
  m = exp.policy.update(experience_batch)
  
  eps_rew += sars.r
  if sars.s1 is None:
    m['r_per_eps'] = eps_rew
    eps_rew = 0
  if step % 100 == 0 or sars.s1 is None:
    for k, v in m.items():
      exp.plt.add_data(k, step, v)
      
# Expect about r_per_eps=-50 in pure random agent.

(1, 1, 32)
QNetwork(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [3]:
opt = grid_world.GridWorld((4, 4)).optimal_expected_reward()
print(f'optimal expected r: {opt}')

optimal expected r: 1.6666666666666665


In [4]:
env.show_model_policy(exp.policy.model)

[[ 0.          0.02276438 -1.06067991 -1.99135995]
 [ 0.03624254 -0.96892565 -1.92039621 -2.79044724]
 [-0.99548662 -2.04977393 -2.72117686 -3.75954938]
 [-2.18253064 -2.86878896 -3.8101747  -4.62500763]]
[['⟲' '←' '←' '←']
 ['↑' '←' '↑' '←']
 ['↑' '↑' '↑' '←']
 ['↑' '↑' '↑' '←']]
[[ 0.15918285  0.         -0.25595942 -1.1751889 ]
 [-0.72936118  0.03398097 -0.85734832 -1.73021519]
 [-1.80981994 -1.0236944  -1.81336808 -2.81641388]
 [-2.92770004 -2.04499412 -2.90813661 -3.61930275]]
[['→' '⟲' '←' '←']
 ['→' '↑' '↑' '←']
 ['→' '↑' '↑' '↑']
 ['→' '↑' '↑' '↑']]
[[-1.96465003 -0.54508442  0.         -0.06489891]
 [-1.93760347 -1.09751606  0.1117529  -1.03832436]
 [-2.7756896  -1.90188372 -0.98539972 -1.76305127]
 [-3.8842721  -2.87123156 -2.03580427 -2.9575429 ]]
[['→' '→' '⟲' '←']
 ['→' '→' '↑' '↑']
 ['↑' '↑' '↑' '↑']
 ['↑' '↑' '↑' '↑']]
[[-1.79167187 -0.77241558  0.12476206  0.        ]
 [-2.64397573 -1.69234884 -0.76052451  0.05139527]
 [-3.63247657 -2.64950395 -1.71738827 -0.82608187]
 

In [5]:
# Debug the action-value function for a given state.
s = [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 2, 0]]
s = np.expand_dims(np.expand_dims(s, 0), -1)
print(exp.policy.model(util.to_variable(s)))
print(exp.policy.target_model(util.to_variable(s)))

tensor([[-1.9843, -1.9797,  0.0113, -1.9095]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([[-1.9261, -2.0092,  0.0131, -1.8275]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)


In [6]:
# Examine the replay buffer.
terminals = rb.filter_by(sars_filter(r=0))
print(len(terminals))

# action-value at 's' was wrong. Search the replay buffer for occurrences of
# the correct transition. It's likely missing, which led the NN to overfit
# on missing data.
trans_s = rb.filter_by(sars_filter(s=s))
trans_s_a = rb.filter_by(sars_filter(s=s,a=2))
print(f'Num with (s): {len(trans_s)}. Num with (s, a): {len(trans_s_a)}')


1680
Num with (s): 75. Num with (s, a): 55


In [7]:
env.visualize(exp.policy, steps=20)

array([[2, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[2, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 2, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 2],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 2],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 2],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 2],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
  